In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
from collections import namedtuple, OrderedDict
import tensorflow as tf
from os import listdir
from os.path import isfile, join
import numpy as np
from PIL import Image
import io
import pandas as pd
from io import BytesIO

In [ ]:
# Read CSV with strategies. 

In [ ]:
main_path = '../input/strategies/train_annotations_strategies.csv'
df_strategies = pd.read_csv(main_path)

In [ ]:
# Select Strategy
STRATEGY = 'C_wbf'

In [ ]:
import os
import pathlib
import tarfile
import re

In [ ]:
!pip install tf_slim
!pip install pycocotools
!pip install lvis
!pip install numba 

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research

# Compile protos.
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
#Environment variables
os.environ['PYTHONPATH'] += ":/kaggle/working/models:/kaggle/working/models/research/slim/:/kaggle/working/models/research:"

import sys
sys.path.append("/kaggle/working/models")
sys.path.append("/kaggle/working/models/research")

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import dataset_util

In [ ]:
train_with_findings = df_strategies[df_strategies['strategy'] == STRATEGY].copy()

# Read the Label Map
label_map = '../input/labelmap/label_map.pbtxt'
label_dict = label_map_util.get_label_map_dict(label_map)

In [ ]:
def split(df, group):
    data = namedtuple('data', ['image_id', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path, label_dict):
    
    # Read the file (dicom)
    data = tf.io.read_file(os.path.join(path, '{}.jpg'.format(group.image_id)))
    image = tf.io.decode_jpeg(data)
    height, width, length = image.shape
    
    # Read the file (jpg)
    #with tf.gfile.GFile(os.path.join(path, '{}.jpg'.format(group.image_id)), 'rb') as fid:
    #    encoded_jpg = fid.read()
        
    #encoded_jpg_io = io.BytesIO(encoded_jpg)
    #image = Image.open(encoded_jpg_io)
    #width, height = image.size
    
    filename = group.image_id.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['x_min'] / width)
        xmaxs.append(row['x_max'] / width)
        ymins.append(row['y_min'] / height)
        ymaxs.append(row['y_max'] / height)
        classes_text.append(row['class_name'].encode('utf8'))
        # Changed to map the Label map file
        # row['class'] contains other label as the labelmap (+ 1)
        classes.append(label_dict[row['class_name']])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(data.numpy()),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))

    return tf_example

In [ ]:
grouped = split(train_with_findings, 'image_id')
splitfactor_train_val = 0.8 # splitfactor_train_val train_share; 1-splitfactor_train_val=val_share

grouped_train = grouped[:int(len(grouped) * splitfactor_train_val)]
grouped_val = grouped[int(len(grouped) * splitfactor_train_val):]
assert len(grouped_val) + len(grouped_train) == len(grouped)

In [ ]:
# Path of the TF Records
tfrecords_train_path = './tfrecords/{strategy}/train'.format(strategy=STRATEGY)
tfrecords_val_path = './tfrecords/{strategy}/val'.format(strategy=STRATEGY)

In [ ]:
if not os.path.exists(tfrecords_train_path):
    os.makedirs(tfrecords_train_path)
if not os.path.exists(tfrecords_val_path):
    os.makedirs(tfrecords_val_path)

In [ ]:
# Generating TRAINING TF Records
filename = '{path}/train.tfrecord'.format(path = tfrecords_train_path)
writer = tf.io.TFRecordWriter(filename)

for group in grouped_train:    
    tf_example = create_tf_example(group, '../input/vinbigdata-original-image-dataset/vinbigdata/train/', label_dict)
    writer.write(tf_example.SerializeToString())
writer.close()

In [ ]:
# Generating TRAINING TF Records
filename = '{path}/val.tfrecord'.format(path = tfrecords_val_path)
writer = tf.io.TFRecordWriter(filename)

for group in grouped_val:    
    tf_example = create_tf_example(group, '../input/vinbigdata-original-image-dataset/vinbigdata/train/', label_dict)
    writer.write(tf_example.SerializeToString())
writer.close()